# Random Forest

In [780]:
import pandas as pd
import numpy as np

# import du csv
data = pd.read_csv('csv_correction.csv')
data

,longitude,latitude,clc_quartier,clc_secteur,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_stadedev,fk_port,fk_pied,fk_situation,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,fk_nomtech,villeca,feuillage,remarquable
0,3.293264,49.840500,Quartier du Centre-Ville,Quai Gayant,6.0,2.0,37.0,EN PLACE,Jeune,semi libre,gazon,Alignement,Non,15.0,5.0,0.0,QUERUB,VILLE,Feuillu,Non
1,3.273380,49.861409,Quartier du Vermandois,Stade Cepy,13.0,1.0,160.0,EN PLACE,Adulte,semi libre,gazon,Groupe,Non,50.0,10.0,0.0,PINNIGnig,VILLE,Conifère,Non
2,3.289068,49.844513,Quartier du Centre-Ville,Rue Villebois Mareuil,12.0,3.0,116.0,REMPLACÉ,Adulte,semi libre,gazon,Alignement,Non,30.0,10.0,0.0,ACEPSE,VILLE,Feuillu,Non
3,3.302387,49.861778,Quartier de l'Europe,Square des Marronniers,16.0,3.0,150.0,EN PLACE,Adulte,semi libre,gazon,Groupe,Non,50.0,2.0,0.0,ACEPLA,VILLE,Feuillu,Non
4,3.304047,49.858446,Quartier de l'Europe,Avenue Buffon,5.0,2.0,170.0,Essouché,Adulte,réduit,gazon,Isolé,Non,40.0,2.0,0.0,SALBAB,VILLE,Feuillu,Non
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7404,3.283505,49.848695,Quartier du Centre-Ville,Place Edouard Branly,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Isolé,Non,5.0,1.0,0.0,ALBJUL,VILLE,Feuillu,Non
7405,3.283436,49.848682,Quartier du Centre-Ville,Place Edouard Branly,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Isolé,Non,3.0,1.0,0.0,ALBJULcho,VILLE,Feuillu,Non
7406,3.297078,49.860413,Quartier de l'Europe,rue Hertz,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Isolé,Non,5.0,1.0,0.0,CEDLIB,VILLE,Conifère,Non
7407,3.291685,49.848009,Quartier du Centre-Ville,Square rue Fréreuse,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Alignement,Non,5.0,1.0,0.0,FAGSYLdawgol,VILLE,Feuillu,Non


In [782]:
data["fk_arb_etat"].value_counts()

fk_arb_etat
EN PLACE        6645
SUPPRIMÉ         394
Essouché         165
REMPLACÉ         137
ABATTU            41
Non essouché      27
Name: count, dtype: int64

In [748]:
from sklearn import preprocessing, feature_selection, model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import cross_val_predict, cross_val_score
import time


In [749]:
data['fk_arb_etat_bool'] = data['fk_arb_etat'].replace(['ABATTU', 'EN PLACE', 'Non essouché', 'REMPLACÉ','SUPPRIMÉ'],'non essouché')

In [750]:
data

,longitude,latitude,clc_quartier,clc_secteur,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_stadedev,fk_port,...,fk_situation,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,fk_nomtech,villeca,feuillage,remarquable,fk_arb_etat_bool
0,3.293264,49.840500,Quartier du Centre-Ville,Quai Gayant,6.0,2.0,37.0,EN PLACE,Jeune,semi libre,...,Alignement,Non,15.0,5.0,0.0,QUERUB,VILLE,Feuillu,Non,non essouché
1,3.273380,49.861409,Quartier du Vermandois,Stade Cepy,13.0,1.0,160.0,EN PLACE,Adulte,semi libre,...,Groupe,Non,50.0,10.0,0.0,PINNIGnig,VILLE,Conifère,Non,non essouché
2,3.289068,49.844513,Quartier du Centre-Ville,Rue Villebois Mareuil,12.0,3.0,116.0,REMPLACÉ,Adulte,semi libre,...,Alignement,Non,30.0,10.0,0.0,ACEPSE,VILLE,Feuillu,Non,non essouché
3,3.302387,49.861778,Quartier de l'Europe,Square des Marronniers,16.0,3.0,150.0,EN PLACE,Adulte,semi libre,...,Groupe,Non,50.0,2.0,0.0,ACEPLA,VILLE,Feuillu,Non,non essouché
4,3.304047,49.858446,Quartier de l'Europe,Avenue Buffon,5.0,2.0,170.0,Essouché,Adulte,réduit,...,Isolé,Non,40.0,2.0,0.0,SALBAB,VILLE,Feuillu,Non,Essouché
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7404,3.283505,49.848695,Quartier du Centre-Ville,Place Edouard Branly,3.0,2.0,15.0,EN PLACE,Jeune,Libre,...,Isolé,Non,5.0,1.0,0.0,ALBJUL,VILLE,Feuillu,Non,non essouché
7405,3.283436,49.848682,Quartier du Centre-Ville,Place Edouard Branly,3.0,2.0,15.0,EN PLACE,Jeune,Libre,...,Isolé,Non,3.0,1.0,0.0,ALBJULcho,VILLE,Feuillu,Non,non essouché
7406,3.297078,49.860413,Quartier de l'Europe,rue Hertz,3.0,2.0,15.0,EN PLACE,Jeune,Libre,...,Isolé,Non,5.0,1.0,0.0,CEDLIB,VILLE,Conifère,Non,non essouché
7407,3.291685,49.848009,Quartier du Centre-Ville,Square rue Fréreuse,3.0,2.0,15.0,EN PLACE,Jeune,Libre,...,Alignement,Non,5.0,1.0,0.0,FAGSYLdawgol,VILLE,Feuillu,Non,non essouché


## Grid Search

In [751]:
# 'fk_arb_etat_bool',
list_col = ['fk_stadedev', 'fk_arb_etat',  'clc_quartier', 'clc_secteur', 'feuillage', 'fk_port', 'fk_situation', 'fk_nomtech']
for col in list_col:
  encoder = preprocessing.LabelEncoder()
  fk_stadedev_encoded = encoder.fit_transform(data[col])
  #print(encoder.categories_)
  data[col] = fk_stadedev_encoded

Split des données

In [752]:
all_cols_list = ["haut_tot", 'haut_tronc', 'tronc_diam', 'fk_stadedev', 'age_estim', 'clc_quartier', 'clc_secteur', 'feuillage', 'fk_port', 'fk_situation', 'fk_nomtech']

In [753]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(data[all_cols_list], data["fk_arb_etat_bool"], test_size=0.2, random_state=42)

In [754]:
# train
classifier = RandomForestClassifier(random_state=42, n_jobs=-1)
params = {"n_estimators": [50, 100, 150, 200, 250, 300]}
gscv = GridSearchCV(estimator=classifier, param_grid=params, scoring="accuracy", cv=5, n_jobs=-1)
classifier_with_grid_search = gscv.fit(X_train, y_train)  # le meilleur modele

print(classifier_with_grid_search.best_params_)
from sklearn.metrics import accuracy_score
cvs = cross_val_predict(gscv, X_train, y_train, cv = 3, n_jobs=-1)

best_n = classifier_with_grid_search.best_params_["n_estimators"]
best_n




{'n_estimators': 250}


250

In [755]:
confusion_matrix(y_train, cvs, normalize="true")

array([[0.15      , 0.85      ],
       [0.00345602, 0.99654398]])

On constate que la matrice de confusion est pas bonne, pas du tout satisfaisante

In [756]:
# TEST (meme si pas besoion car modele pas validé)
# y_test_predicted = classifier_with_grid_search.predict(X_test)
# accuracy_score(y_test, y_test_predicted)

## Feature Selection

In [757]:
all_cols_list = ["haut_tot", 'haut_tronc', 'tronc_diam', 'fk_stadedev', 'age_estim', 'clc_quartier', 'clc_secteur', 'feuillage', 'fk_port', 'fk_situation', 'fk_nomtech']

In [758]:
# 'fk_arb_etat_bool', 
list_col = ['fk_stadedev', 'fk_arb_etat', 'clc_quartier', 'clc_secteur', 'feuillage', 'fk_port', 'fk_situation', 'fk_nomtech']
for col in list_col:
  encoder = preprocessing.LabelEncoder()
  fk_stadedev_encoded = encoder.fit_transform(data[col])
  #print(encoder.categories_)
  data[col] = fk_stadedev_encoded

In [759]:
# Train the Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model
accuracy_before = rf.score(X_test, y_test)

In [760]:
accuracy_before

0.979757085020243

In [761]:
importances = rf.feature_importances_
feature_names = data[all_cols_list].columns

feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importance_df

,Feature,Importance
0,haut_tot,0.151611
1,haut_tronc,0.112041
2,tronc_diam,0.268431
3,fk_stadedev,0.013490
4,age_estim,0.073595
5,clc_quartier,0.060569
6,clc_secteur,0.129609
7,feuillage,0.011906
8,fk_port,0.024605
9,fk_situation,0.030371


In [762]:
# Rank features by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)

         Feature  Importance
2     tronc_diam    0.268431
0       haut_tot    0.151611
6    clc_secteur    0.129609
10    fk_nomtech    0.123771
1     haut_tronc    0.112041
4      age_estim    0.073595
5   clc_quartier    0.060569
9   fk_situation    0.030371
8        fk_port    0.024605
3    fk_stadedev    0.013490
7      feuillage    0.011906


In [763]:
# Select top N features (example selecting top 5 features)
top_features = feature_importance_df['Feature'][:5].values
X_train_selected = X_train[top_features]
X_test_selected = X_test[top_features]

In [764]:
# Train the Random Forest model with selected features
start_time = time.time()
rf_selected = RandomForestClassifier(n_estimators=best_n, random_state=42, n_jobs=-1)
rf_selected.fit(X_train_selected, y_train)
end_time = time.time()
y_test_predicted = rf_selected.predict(X_test_selected)


In [765]:
# validation
from sklearn.metrics import accuracy_score
cvs = cross_val_predict(rf_selected, X_train, y_train, cv = 3, n_jobs=-1)
confusion_matrix(y_train, cvs, normalize="true")


array([[0.15      , 0.85      ],
       [0.00345602, 0.99654398]])

In [766]:
# TEST
#accuracy_after = rf_selected.score(X_test_selected, y_test)
print(f'Accuracy after feature selection: {accuracy_score(y_test, y_test_predicted):.3f}')

print(f'Temps écoulé : {end_time - start_time :.3f} secondes.')

Accuracy after feature selection: 0.980
Temps écoulé : 1.816 secondes.


Résultats de matrice de confusion très mauvais, trop grand déséquilibre entre les essouchés et les non essouchés

# SMOTE

On va utiliser SMOTE pour ajouter artificiellement des essouchés

In [767]:
from imblearn.over_sampling import SMOTE

In [768]:
data

,longitude,latitude,clc_quartier,clc_secteur,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_stadedev,fk_port,...,fk_situation,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,fk_nomtech,villeca,feuillage,remarquable,fk_arb_etat_bool
0,3.293264,49.840500,7,112,6.0,2.0,37.0,1,1,8,...,0,Non,15.0,5.0,0.0,185,VILLE,1,Non,non essouché
1,3.273380,49.861409,8,228,13.0,1.0,160.0,1,0,8,...,1,Non,50.0,10.0,0.0,129,VILLE,0,Non,non essouché
2,3.289068,49.844513,7,166,12.0,3.0,116.0,4,0,8,...,0,Non,30.0,10.0,0.0,20,VILLE,1,Non,non essouché
3,3.302387,49.861778,6,225,16.0,3.0,150.0,1,0,8,...,1,Non,50.0,2.0,0.0,15,VILLE,1,Non,non essouché
4,3.304047,49.858446,6,4,5.0,2.0,170.0,2,0,6,...,2,Non,40.0,2.0,0.0,192,VILLE,1,Non,Essouché
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7404,3.283505,49.848695,7,103,3.0,2.0,15.0,1,1,1,...,2,Non,5.0,1.0,0.0,30,VILLE,1,Non,non essouché
7405,3.283436,49.848682,7,103,3.0,2.0,15.0,1,1,1,...,2,Non,3.0,1.0,0.0,31,VILLE,1,Non,non essouché
7406,3.297078,49.860413,6,238,3.0,2.0,15.0,1,1,1,...,2,Non,5.0,1.0,0.0,50,VILLE,0,Non,non essouché
7407,3.291685,49.848009,7,226,3.0,2.0,15.0,1,1,1,...,0,Non,5.0,1.0,0.0,70,VILLE,1,Non,non essouché


Split des données en train et test

In [769]:
sm = SMOTE(random_state=42, sampling_strategy=0.15)
X_smoted, y_smoted = sm.fit_resample(X_train_selected, y_train)

In [770]:
X_smoted

,tronc_diam,haut_tot,clc_secteur,fk_nomtech,haut_tronc
0,110.000000,13.000000,90,20,2.000000
1,115.000000,13.000000,34,94,2.000000
2,113.000000,16.000000,82,132,5.000000
3,165.000000,14.000000,10,45,2.000000
4,80.000000,8.000000,14,169,3.000000
...,...,...,...,...,...
6650,197.759049,22.931086,90,212,6.069163
6651,94.037108,11.186541,168,24,0.000000
6652,189.837312,21.814071,89,213,2.494190
6653,208.551062,24.782659,91,15,13.927553


In [771]:
y_smoted

0       non essouché
1       non essouché
2       non essouché
3       non essouché
4       non essouché
            ...     
6650        Essouché
6651        Essouché
6652        Essouché
6653        Essouché
6654        Essouché
Name: fk_arb_etat_bool, Length: 6655, dtype: object

In [772]:
classifier = RandomForestClassifier(random_state=42, n_jobs=-1, n_estimators=best_n)
classifier_with_grid_search_and_FS_and_SMOTE = classifier.fit(X_smoted, y_smoted)  # le meilleur modele

Validation

In [773]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict, cross_val_score

In [774]:
cvs = cross_val_predict(gscv, X_smoted, y_smoted, cv = 3, n_jobs=-1)
confusion_matrix(y_smoted, cvs, normalize="true")

array([[0.77419355, 0.22580645],
       [0.00846725, 0.99153275]])

In [775]:
np.mean(cross_val_score(classifier, X_smoted, y_smoted, cv = 3, n_jobs=-1))

0.9643905213506384

La matrice de confusion est satisfaisante, on passe aux tests.

In [776]:
from sklearn.metrics import accuracy_score

y_test_predicted = classifier_with_grid_search_and_FS_and_SMOTE.predict(X_test_selected)
accuracy_score(y_test, y_test_predicted)

0.979757085020243

In [777]:
y_pred = classifier_with_grid_search_and_FS_and_SMOTE.predict(X_test_selected)
confusion_matrix(y_test, y_pred, normalize="true")

array([[0.16      , 0.84      ],
       [0.00617708, 0.99382292]])

In [778]:
y_pred = classifier_with_grid_search_and_FS_and_SMOTE.predict(X_test_selected)
y_proba = classifier_with_grid_search_and_FS_and_SMOTE.predict_proba(X_test_selected)

# Afficher la matrice de confusion et le rapport de classification
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))



[[   4   21]
 [   9 1448]]
              precision    recall  f1-score   support

    Essouché       0.31      0.16      0.21        25
non essouché       0.99      0.99      0.99      1457

    accuracy                           0.98      1482
   macro avg       0.65      0.58      0.60      1482
weighted avg       0.97      0.98      0.98      1482



In [783]:
"""
# Créer un DataFrame avec les résultats
results = X_test.copy()
results['Actual'] = y_test
results['Predicted'] = y_pred
results['Probability_False'] = y_proba[:, 0]
results['Probability_True'] = y_proba[:, 1]
  
# Afficher les premières lignes du DataFrame
print(results.head())"""

"\n# Créer un DataFrame avec les résultats\nresults = X_test.copy()\nresults['Actual'] = y_test\nresults['Predicted'] = y_pred\nresults['Probability_False'] = y_proba[:, 0]\nresults['Probability_True'] = y_proba[:, 1]\n  \n# Afficher les premières lignes du DataFrame\nprint(results.head())"